# 01 Starter: Loading and Viewing the NWB Data

This notebook walks through the NWB data loading and conversion tools, as well as providing some simple visualizations for LINK dataset.

### 1. Download LINK Dataset

We need to first install DANDI-cli tool for downloading the dataset. By default (of the series of tutorial notebooks) it downloads data to the root directory.

In [ ]:
# Install dandi
%pip install dandi
# Download LINK dataset using dandi

!dandi download DANDI:001201 -o ../../

Double check that data is downloaded to the root directory by examine that if a folder called 001201 exists with various nwb files in its sub-Monkey-N subfolder. Run the following cell, there should be 312 NWB files found.

In [ ]:
import os
dataset_dir = os.path.join('..', '..', '001201')
subfolder = os.path.join(dataset_dir, 'sub-Monkey-N')

if os.path.isdir(dataset_dir) and os.path.isdir(subfolder):
    nwb_files = [f for f in os.listdir(subfolder) if f.endswith('.nwb')]
    if nwb_files:
        print(f"Data is downloaded. Found {len(nwb_files)} NWB files in {subfolder}:")
    else:
        print(f"No NWB files found in {subfolder}.")
else:
    print(f"Dataset folder or subfolder not found at {subfolder}.")

### 2. Accessing NWB Files Directly Using PyNWB


We will need to install pynwb library for accessing NWB files in python.

In [ ]:
# Install pynwb
%pip install -U pynwb

We import pynwb here and look at the overall structure for the first nwb file. [TODO: some help here describing what NWB file contains]

In [ ]:
from pynwb import NWBHDF5IO

dataset_dir = os.path.join('..', '..', '001201')
subfolder = os.path.join(dataset_dir, 'sub-Monkey-N')

nwb_path = os.path.join(subfolder, nwb_files[0])
with NWBHDF5IO(nwb_path, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)

Here we examine a simple case of neural and behavioural data.

To look at the neural data, analysis section needs to be accessed. For neural data, it contains spiking band power and threshold crossing. For behavioural data, it contains index position, MRS position, index velocity, and MRS velocity. 

In this NWB file, each type of neural data contains a matrix of (27649, 96), corresponding to (n_TimeBins, n_Channels) in general. For each type of behavioural data, it contains a matrix of (27649, 1).

[TODO: talk about what unit is in SBP and why /0.25]

The following cell access the analysis section of the NWB file and organises neural data into a ndarray called sbp and behavioural data into a ndarray called kinematic_names.

In [ ]:
import numpy as np
kinematic_names =   (
                    "index_position",
                    "mrs_position",
                    "index_velocity",
                    "mrs_velocity",
                    )

with NWBHDF5IO(nwb_path, 'r') as io:
    nwbfile = io.read()
    sbp = (nwbfile.analysis["SpikingBandPower"].data[:] / 0.25).astype(np.float64)
    
    finger_kinematics = np.column_stack(
                                        [nwbfile.analysis[f"{n}"].data[:].ravel() 
                                        for n in kinematic_names]
                                        ).astype(np.float64)


Here is a simple plot to demonstrate what these data look like. Channel 30 is selected from the SBP data arbitrarily.

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(5, 1, figsize=(12, 10), sharex=True)
axs[0].plot(sbp[:,30])
axs[0].set_ylabel('Spiking Band Power')

for i in range(4):
    axs[i+1].plot(finger_kinematics[:,i])
    axs[i+1].set_ylabel(f'{kinematic_names[i]}')

axs[-1].set_xlabel('Time')
plt.tight_layout()
plt.show()

### 2. Converting NWB Files Using nwb_tools

nwb_tools.py is provided for easier access and conversion of NWB files to downstream tasks. We start by importing nwb_tools.

In [ ]:
# Setting working directory to root to mimic usual usecase
###############################################################
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, root_path)
###############################################################

from tutorial.utils import nwb_tools

load_one_nwb tool converts one nwb file into dictionary, as shown below,

In [ ]:
dict_from_nwb = nwb_tools.load_one_nwb(os.path.join(subfolder, nwb_files[0]))
print(dict_from_nwb)

Here is another tool to convert all NWB data into pkl files for downstream applications.

[TODO: talk about what pickle files contain]

In [ ]:
data_path = dataset_dir
output_path = os.path.join(dataset_dir, '../data_test')
if not os.path.exists(output_path):
    os.makedirs(output_path)

nwb_tools.convert_dandi_nwb_to_pkl(data_path, output_path)